In [ ]:
import json
import numpy as np
from pyquaternion import Quaternion

In [ ]:
import pickle

In [ ]:
import open3d as o3d
import plotly.io as pio
import plotly.graph_objects as go
import os.path as osp
import os
import math
import matplotlib.pyplot as plt
import time
import csv

In [ ]:
o3d.__version__

In [ ]:
import copy

In [ ]:
data_dir = "/media/kluo/My Passport/v2x_dataset"

scene_id = 18


scene_infos = []
with open(osp.join(data_dir, "merged_info_updated", f"info_{scene_id}.jsonl"), "r") as file:
    for line in file:
        scene_infos.append(json.loads(line))
len(scene_infos), scene_infos[0]

In [ ]:
# check number of labeled timeframes matches
sum([info['labeled'] for info in scene_infos])

## Labels for the scene

In [ ]:
with open(osp.join(data_dir, "scene18_refined_annos.pkl"), "rb") as f:
    per_sensor_labels = pickle.load(f)
per_sensor_labels.keys()

In [ ]:
# check yaws are the same???
def quaternion_yaw(q: Quaternion) -> float:
    """
    Calculate the yaw angle from a quaternion.
    Note that this only works for a quaternion that represents a box in lidar or global coordinate frame.
    It does not work for a box in the camera frame.
    :param q: Quaternion of interest.
    :return: Yaw angle in radians.
    """

    # Project into xy plane.
    v = np.dot(q.rotation_matrix, np.array([1, 0, 0]))

    # Measure yaw using arctan.
    yaw = np.arctan2(v[1], v[0])

    return yaw

In [ ]:
category_to_color_map = {
    1: [1., 0.8, 0.9], #car
    2: [0.8, 0.8, 1.], #ped
    3: [1., 1., 1.], #truck
    4: [0.8, 1., 0.8], #bike
}

In [ ]:
per_sensor_labels['top'][0]

## Make a quick video

In [ ]:
view_status_dometop = """{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 218.3385009765625, 76.248146057128906, 20.960601806640625 ],
			"boundingbox_min" : [ -180.83258056640625, -149.74227905273438, -11.554059028625488 ],
			"field_of_view" : 60.0,
			"front" : [ 0.14969908237785134, -0.80852446648900111, 0.56910312933940677 ],
			"lookat" : [ -33.459614649965793, -65.170023496217837, -3.7534205763743964 ],
			"up" : [ -0.072208736619961217, 0.56511189561377617, 0.82184818780085778 ],
			"zoom" : 0.059999999999999609
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}"""
view_status_003 = """{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 143.39578, 39.981537000000003, 20.3521 ],
			"boundingbox_min" : [ -77.780579000000003, -26.723534000000001, -2.9170704000000001 ],
			"field_of_view" : 60.0,
			"front" : [ -0.8788139935861029, 0.064951891427763711, 0.47272319223537279 ],
			"lookat" : [ -27.450282764100578, 1.4187824521767032, 35.302548704627732 ],
			"up" : [ 0.47615991748266312, 0.055121053390291934, 0.87762942205468952 ],
			"zoom" : 0.02
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}"""

view_status_004 = """{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 115.07069, 45.020473000000003, 17.247612 ],
			"boundingbox_min" : [ -59.609820999999997, -65.848395999999994, -2.9546332 ],
			"field_of_view" : 60.0,
			"front" : [ -0.84668236447518408, 0.0089610073709900182, 0.53202318937580972 ],
			"lookat" : [ 5.8921459746849258, -0.40136187399394252, 5.3581494394887308 ],
			"up" : [ 0.53205120823209162, 0.00090611543299307616, 0.84671169282914349 ],
			"zoom" : 0.12000000000000001
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}"""

view_status_by_sensor = {
    "top": view_status_dometop,
    "dome": view_status_dometop,
    "003": view_status_003,
    "004": view_status_004
}

In [ ]:
def lerp(start, end, step, total_steps):
    return start + (end - start) * (step / total_steps)

def interpolate_annotations(label_frames_data, freq=10.):
    interpolated_labels_data = {}
    timesteps = list(label_frames_data.keys())
    timesteps.sort()
    for i in timesteps[:-1]:
        current_frame = label_frames_data[i]
        next_frame = label_frames_data[i + 10]
        interpolated_labels_data[i] = current_frame
        
        # Build a map of next positions
        next_positions = {obj['track_id']: obj for obj in next_frame}
        offs = 1.7e18  #offset for numerical stability
        for j in range(1, freq):
            interpolated_frame = {}
            
            interp_annotations = []
            # Only interpolate if the object exists in both current and next frame
            for obj in current_frame:
                if obj['track_id'] in next_positions:
                    interp_obj = copy.deepcopy(obj)
                    next_obj = next_positions[obj['track_id']]
                    
                    # update is_keyframe, position, rotation, yaw
                    interp_obj['is_keyframe'] = False
                    
                    interp_obj['position']['x'] = lerp(obj['position']['x'], next_obj['position']['x'], j, freq)
                    interp_obj['position']['y'] = lerp(obj['position']['y'], next_obj['position']['y'], j, freq)
                    interp_obj['position']['z'] = lerp(obj['position']['z'], next_obj['position']['z'], j, freq)
                    
                    obj_r = Quaternion(obj['rotation']['qw'], obj['rotation']['qx'], obj['rotation']['qy'], obj['rotation']['qz'],)
                    next_r = Quaternion(next_obj['rotation']['qw'], next_obj['rotation']['qx'], next_obj['rotation']['qy'], next_obj['rotation']['qz'],)
                    q  = Quaternion.slerp(obj_r, next_r, j / freq) 
                    qw, qx, qy, qz = q.elements
                    interp_obj['rotation']['qx'] = qx
                    interp_obj['rotation']['qy'] = qy
                    interp_obj['rotation']['qz'] = qz
                    interp_obj['rotation']['qw'] = qw
                    interp_obj['yaw'] = quaternion_yaw(q)
                    
                    interp_annotations.append(interp_obj)
            interpolated_frame["annotations"] = interp_annotations
            # interpolated_labels_data.append(interpolated_frame)
            interpolated_labels_data[i + j] = interp_annotations
                    
    interpolated_labels_data[timesteps[-1]] = next_frame  # Add the last frame
    return interpolated_labels_data

In [ ]:
interpolated_label_frames_sensor = {}
for sensor, label_frames in per_sensor_labels.items():
    interpolated_label_frames = interpolate_annotations(label_frames, 10)
    interpolated_label_frames_sensor[sensor] = interpolated_label_frames

In [ ]:
len(interpolated_label_frames_sensor['top']), len(interpolated_label_frames_sensor['dome']), len(interpolated_label_frames_sensor['004'])

In [ ]:
interpolated_label_frames_sensor['top'].keys()

In [ ]:
interpolated_label_frames_sensor['003'].keys()

In [ ]:
with open(osp.join(data_dir, "scene_18_per_frame_annos.pkl"), "rb") as f:
    loaded_labels_per_sensor = pickle.load(f)
len(loaded_labels_per_sensor['003'])

In [ ]:
# remap for easier access
loaded_labels_per_sensor_by_idx = {}
for sensor, annos in loaded_labels_per_sensor.items():
    loaded_labels_per_sensor_by_idx[sensor] = {anno['timestep']: anno['annotations'] for anno in annos}
loaded_labels_per_sensor_by_idx['003'].keys()

In [ ]:
for ts in [186, 187, 188, 189]:
    interpolated_label_frames_sensor['003'][ts] = loaded_labels_per_sensor_by_idx['003'][ts]
interpolated_label_frames_sensor['003'].keys()

In [ ]:
# visualize over scene
timestep = 200
sensor = '004'

annos = interpolated_label_frames_sensor[sensor][timestep]
info = scene_infos[timestep]
annos, info

In [ ]:
info = scene_infos[timestep][sensor]
pcd_filepath = osp.join("/media/kluo/My Passport/", info['pcd_filepath'])
pc = o3d.io.read_point_cloud(pcd_filepath)
pc.paint_uniform_color(np.array([0, 102, 255]) / 255.)
points = np.array(pc.points)
points.shape, pcd_filepath

In [ ]:
# convert annotations
samples = []
print(timestep)
for i in range(len(annos)):
    category = annos[i]['category_id']
    size = np.array([annos[i]['dimensions']['x'], annos[i]['dimensions']['y'], annos[i]['dimensions']['z']])
    center = np.array([annos[i]['position']['x'], annos[i]['position']['y'], annos[i]['position']['z']])
    rotation = Quaternion(annos[i]['rotation']['qw'], annos[i]['rotation']['qx'],annos[i]['rotation']['qy'],annos[i]['rotation']['qz'],)
    r = rotation.rotation_matrix
    sample_3d = o3d.geometry.OrientedBoundingBox(center, r, size)
    sample_3d.color = [0,0,0]  #category_to_color_map[int(category)]
    samples.append(sample_3d)
    
o3d.visualization.draw_geometries([pc] + samples)

In [ ]:
sensor_to_rgb_map = {'003': np.array([204, 102, 153]), 
                     '004': np.array([255, 153, 51]),
                     'laser': np.array([255, 255, 153]), 
                     'top': np.array([0, 102, 255]), 
                     'dome': np.array([0, 204, 102])}

In [ ]:
# GENERATE VIDEO AT HIGHER FREQ
for sensor in ['dome']:
    save_dir = f"./video_vis_interpolated_{sensor}/"
    timesteps = [13, 72]

#     timesteps = list(loaded_labels_per_sensor_by_idx[sensor])
    timesteps.sort()

    for timestep in timesteps:
        print(timestep)
        vis = o3d.visualization.Visualizer()
        vis.create_window()

        vis.set_full_screen(True)
        vis.get_render_option().background_color = [0.01, 0, 0]
        vis.get_render_option().point_size = 1.5

        annos = loaded_labels_per_sensor_by_idx[sensor][timestep]
        info = scene_infos[timestep][sensor]

        # load points
        pcd_filepath = osp.join("/media/kluo/My Passport/", info['pcd_filepath'])
        pc = o3d.io.read_point_cloud(pcd_filepath)
        pc.paint_uniform_color(sensor_to_rgb_map[sensor] / 255.)
        vis.add_geometry(pc)  # add point cloud
    #     timestep = info['dome']['pcd_filepath'].split("/")[-1].split("_")[1]
    #     merged_pcd_filepath = osp.join(data_dir, f"MergedPointCloud/mini_{scene_id}/", f"merged_{timestep}.pcd")
    #     merged_pc = o3d.io.read_point_cloud(merged_pcd_filepath)
    #     vis.add_geometry(merged_pc)  # add point cloud

        # convert annotations
        for i in range(len(annos)):
            category = annos[i]['category_id']
            size = np.array([annos[i]['dimensions']['x'], annos[i]['dimensions']['y'], annos[i]['dimensions']['z']])
            center = np.array([annos[i]['position']['x'], annos[i]['position']['y'], annos[i]['position']['z']])
            rotation = Quaternion(annos[i]['rotation']['qw'], annos[i]['rotation']['qx'],annos[i]['rotation']['qy'],annos[i]['rotation']['qz'],)
            r = rotation.rotation_matrix
            sample_3d = o3d.geometry.OrientedBoundingBox(center, r, size)
            sample_3d.color = category_to_color_map[int(category)]
            vis.add_geometry(sample_3d)

        # add or update geometry objects
        # vis.add_geometry([merged_pc] + samples)  # add point cloud
        vis.set_view_status(view_status_by_sensor[sensor])
        # update o3d window
        if not vis.poll_events():
            break
        vis.update_renderer()
        time.sleep(2.)
        vis.capture_screen_image(osp.join(save_dir, "%03d.jpg" % timestep))
        time.sleep(0.5)

        vis.close()
        vis.destroy_window()

# Transform bounding boxes into each frame

In [ ]:
def translate_and_rotate_position(p_o, p_s, q_s):
    """Translate and rotate position of object O relative to system S."""
    # Translate
    p_relative = np.array(p_o) - np.array(p_s)
    
    # Rotate using the inverse of q_s
    q_s_inv = q_s.inverse
    p_rotated = q_s_inv.rotate(p_relative)
    # p_rotated = rotate_vector(p_relative, q_s_inv)
    
    return p_rotated

def relative_orientation(q_o, q_s):
    """Calculate the relative orientation of O with respect to S."""
    return q_s.inverse * q_o

# Example usage:
p_o = (1, 2, 3)  # Position of object O
p_s = (4, 5, 6)  # Position of system S
q_o = Quaternion(axis=[0, 1, 0], angle=np.pi/2)  # Orientation of object O
q_s = Quaternion(axis=[0, 0, 1], angle=np.pi/4)  # Orientation of system S

# Convert position
new_position = translate_and_rotate_position(p_o, p_s, q_s)
print("New position:", new_position)

# Convert orientation
new_orientation = relative_orientation(q_o, q_s)
print("New orientation:", new_orientation)

In [ ]:
# w, x, y, z
type(new_orientation), new_orientation.elements

In [ ]:
len(interpolated_label_frames)

In [ ]:
interpolated_label_frames[0]

In [ ]:
# info['003'], info['top']
info['top']

In [ ]:
annos

In [ ]:
labels_per_sensor = {}
# for each frame
# for each sensor
# convert label object to sensor coord
# concatenate
# save scene_info idx
for timestep in range(len(interpolated_label_frames)):
    annos = interpolated_label_frames[timestep]['annotations']
    info = scene_infos[timestep]
    
    for key, val in info.items():
        if key == "labeled":
            continue
        # else, key is a sensor
        transl_s = np.array([val["pose_translation"]['x'], val["pose_translation"]['y'], val["pose_translation"]['z']])
        rota_s = Quaternion(val["pose_rotation"]['w'], val["pose_rotation"]['x'], val["pose_rotation"]['y'], val["pose_rotation"]['z'])
        
        sensor_annos = copy.deepcopy(annos)
        # convert annotations
        for i in range(len(sensor_annos)):
            center_o = np.array([sensor_annos[i]['position']['x'], sensor_annos[i]['position']['y'], sensor_annos[i]['position']['z']])
            rota_o = Quaternion(sensor_annos[i]['rotation']['qw'], sensor_annos[i]['rotation']['qx'], sensor_annos[i]['rotation']['qy'], sensor_annos[i]['rotation']['qz'],)
            new_position = translate_and_rotate_position(center_o, transl_s, rota_s)
            new_orientation = relative_orientation(rota_o, rota_s)
            
            sensor_annos[i]['position']['x'] = new_position[0]
            sensor_annos[i]['position']['y'] = new_position[1]
            sensor_annos[i]['position']['z'] = new_position[2]
            
            qw, qx, qy, qz = new_orientation.elements
            sensor_annos[i]['rotation']['qx'] = qx
            sensor_annos[i]['rotation']['qy'] = qy
            sensor_annos[i]['rotation']['qz'] = qz
            sensor_annos[i]['rotation']['qw'] = qw
            sensor_annos[i]['yaw'] = quaternion_yaw(new_orientation)
        
        if key not in labels_per_sensor:
            labels_per_sensor[key] = [{'annotations': sensor_annos, 'timestep': timestep}]
        else:
            labels_per_sensor[key].append({'annotations': sensor_annos, 'timestep': timestep})

In [ ]:
labels_per_sensor.keys()

In [ ]:
len(labels_per_sensor['004']), labels_per_sensor['004'][0]

In [ ]:
timestep = 200
sensor = 'top'
info = scene_infos[timestep][sensor]
pcd_filepath = osp.join("/media/kluo/My Passport/", info['pcd_filepath'])
pc = o3d.io.read_point_cloud(pcd_filepath)
points = np.array(pc.points)
points.shape, pcd_filepath

In [ ]:
timestep_to_idx_map = {anno['timestep']: i for i, anno in enumerate(labels_per_sensor[sensor])}

In [ ]:
# convert annotations
samples = []
timestep
annos = labels_per_sensor[sensor][timestep_to_idx_map[timestep]]['annotations']
for i in range(len(annos)):
    category = annos[i]['category_id']
    size = np.array([annos[i]['dimensions']['x'], annos[i]['dimensions']['y'], annos[i]['dimensions']['z']])
    center = np.array([annos[i]['position']['x'], annos[i]['position']['y'], annos[i]['position']['z']])
    rotation = Quaternion(annos[i]['rotation']['qw'], annos[i]['rotation']['qx'],annos[i]['rotation']['qy'],annos[i]['rotation']['qz'],)
    r = rotation.rotation_matrix
    sample_3d = o3d.geometry.OrientedBoundingBox(center, r, size)
    sample_3d.color = [0,0,0]  #category_to_color_map[int(category)]
    samples.append(sample_3d)

In [ ]:
o3d.visualization.draw_geometries([pc] + samples)

In [ ]:
import pickle

In [ ]:
with open("scene_18_per_frame_annos.pkl", "wb") as f:
    pickle.dump(labels_per_sensor, f)

In [ ]:
# check it's saved properly

with open("scene_18_per_frame_annos.pkl", "rb") as f:
    loaded_labels_per_sensor = pickle.load(f)

loaded_labels_per_sensor.keys()

In [ ]:
for k, v in loaded_labels_per_sensor.items():
    print(len(v))